# Comeback Coins 

A data storytelling project on first generation (bitcoin) and second generation (ethereum) cryptocurrency.

In [ ]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import bs4 as bs
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
%matplotlib inline

<h2 id="DSCW">Downloading And Scraping The Contents Of A Web Page</h2> 


We Download the contents of the web page:


In [ ]:
url = "http://www.cryptowat.ch"

We use <code>get</code> to download the contents of the webpage in text format and store in a variable called <code>data</code>:


In [ ]:
data  = requests.get(url).text 

We create a <code>BeautifulSoup</code> object using the <code>BeautifulSoup</code> constructor


In [ ]:
soup = BeautifulSoup(data,"html.parser")  # create a soup object using the variable 'data'

Scrape all links


In [ ]:
for link in soup.find_all('a',href=True):  # in html anchor/link is represented by the tag <a>

    print(link.get('href'))


This helps us identify the contents of the webpage and how we can scrape the data. The data we need is from the cryptocurrency markets.

In [ ]:
#let's inspect the elements in the cryptocurrency markets

url = "http://www.cryptowat.ch/markets"

In [ ]:
data  = requests.get(url).text 

In [ ]:
soup = BeautifulSoup(data,"html.parser")

In [ ]:
for link in soup.find_all('a',href=True):  # in html anchor/link is represented by the tag <a>

    print(link.get('href'))

We see there is an information on crypto market for all markets here. Let's inspect the contents.

In [ ]:
url="https://api.cryptowat.ch/markets"

In [ ]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [ ]:
soup = BeautifulSoup(data,"html.parser")

In [ ]:
def formatter(string):
    formatted = bytes(string, 'ascii').decode('unicode-escape')
    # for Python 2
    # formatted = string.decode('unicode-escape')
    return formatted

In [ ]:
print(soup.prettify(formatter = formatter))

There are a lot of exchange market like bitfitnex, kraken, ftx-us, binance, etc. For this I will go with binance.

In [ ]:
#scrape the data by creating a function
def get_crypto_price(symbol, exchange='binance', after='2018-09-01'):
    url = 'https://api.cryptowat.ch/markets/{exchange}/{symbol}usd/ohlc'.format(
        symbol=symbol, exchange=exchange)
    resp = requests.get(url, params={
        'periods': '7200',
        'after': str(int(pd.Timestamp(after).timestamp()))
    })
#     resp.raise_for_status()
    data = resp.json()
    df = pd.DataFrame(data['result']['7200'], columns=[
        'CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'NA'
    ])
    df['CloseTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    df.set_index('CloseTime', inplace=True)
    return df

In [ ]:
#pull data from crypto for btc and eth
btc = get_crypto_price('btc', 'bitstamp')
eth = get_crypto_price('eth', 'bitstamp')

In [ ]:
#pull others for comparison
ada = get_crypto_price('ada', 'bitstamp')
xrp = get_crypto_price('xrp', 'bitstamp')
usdt = get_crypto_price('usdt', 'bitstamp')
matic = get_crypto_price('matic', 'bitstamp')
usdc = get_crypto_price('usdc', 'bitstamp')
shib = get_crypto_price('shib', 'bitstamp')
ftm = get_crypto_price('ftm', 'bitstamp')
uma = get_crypto_price('uma', 'bitstamp')

## Descriptive Statistics

In [ ]:
btc.describe()

In [ ]:
eth.describe()

The dataset is between 1-6000 rows and 6 columns.

## Data Preprocessing and Cleaning

In [ ]:
#export to csv to make the index a column
btc.to_csv('btc.csv', index=True)
eth.to_csv('eth.csv', index=True)
ada.to_csv('ada.csv', index=True)
xrp.to_csv('xrp.csv', index=True)
usdt.to_csv('usdt.csv', index=True)
matic.to_csv('matic.csv', index=True)
usdc.to_csv('usdc.csv', index=True)
shib.to_csv('shib.csv', index=True)
ftm.to_csv('ftm.csv', index=True)
uma.to_csv('uma.csv', index=True)

In [ ]:
#import
df_btc = pd.read_csv('btc.csv')
df_eth = pd.read_csv('eth.csv')
df_ada = pd.read_csv('ada.csv')
df_xrp = pd.read_csv('xrp.csv')
df_usdt = pd.read_csv('usdt.csv')
df_matic = pd.read_csv('matic.csv')
df_usdc = pd.read_csv('usdc.csv')
df_shib = pd.read_csv('shib.csv')
df_ftm = pd.read_csv('ftm.csv')
df_uma = pd.read_csv('uma.csv')

In [ ]:
#assign new column
df_btc['Symbol'] = "BTC"
df_eth['Symbol'] = "ETH"
df_ada['Symbol'] = "ADA"
df_xrp['Symbol'] = "XRP"
df_usdt['Symbol'] = "USDT"
df_matic['Symbol'] = "MATIC"
df_usdc['Symbol'] = "USDC"
df_shib['Symbol'] = "SHIB"
df_ftm['Symbol'] = "FTM"
df_uma['Symbol'] = "UMA"

In [ ]:
#join the dataframe for data comparison later
coins = pd.concat([df_btc, df_eth, df_ada, df_xrp, df_usdt, df_matic, df_usdc, df_shib, df_ftm, df_uma], axis=0)
coins.head()

In [ ]:
#check null values
sns.heatmap(coins.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

Here we see that there are no missing values in the data set

In [ ]:
coins.info()

In [ ]:
coins.duplicated()

In [ ]:
#convert the CloseTime column to datetime format
coins['CloseTime']= pd.to_datetime(coins['CloseTime'])
 
# Check the format of 'Date' column
coins.info()
coins.head()

In [ ]:
#extract year and month into separate column
coins['Year'] = pd.to_datetime(coins['CloseTime']).dt.year
coins['Month'] = pd.to_datetime(coins['CloseTime']).dt.month

In [ ]:
coins.info()

In [ ]:
coins.head()

## EDA

In [ ]:
import matplotlib.dates as mdates
import matplotlib.ticker as mtick

In [ ]:
#use a barchart to view the coin performance
#create a bar chart using Seaborn
sns.set_style("whitegrid")
bar,ax = plt.subplots(figsize=(10,10))
ax = sns.barplot(x='ClosePrice', y='Symbol', data=coins, ci=None, color='grey',orient='h')
ax.set_title("Crypto Prices in USD", fontsize=25)
ax.set(xscale="log")#set scale to view others
ax.set_xlabel ("Crypto", fontsize=20)
ax.set_ylabel (None, fontsize=20)
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
plt.savefig("cryptoprice.svg")


In [ ]:
#check mean performance of all
df=coins.groupby('Symbol').agg({'ClosePrice':np.mean,'OpenPrice':np.mean,'HighPrice':np.mean,'LowPrice':np.mean}).reset_index()
fig = px.line(df, x='Symbol', y=['ClosePrice','HighPrice','LowPrice','OpenPrice'])
fig.update_layout(template='plotly_dark',
    yaxis_title='Price',
    title='Mean OHLC Prices of crypto currencies',
    hovermode="x",
    title_x=0.5
)
fig.update_yaxes( # the y-axis is in dollars
    tickprefix="$", showgrid=True
)
fig.show()

In [ ]:
df.head()

# Analyzing bitcoin prices for 2020

In [ ]:
df_2020 = coins[coins.Year == 2020]
df_2020_btc = df_2020[df_2020.Symbol == 'BTC']
df_2020_btc.head()

In [ ]:
import plotly.graph_objects as go

In [ ]:
#check growth for 2020
fig = go.Figure(data=go.Ohlc(x=df_2020_btc['CloseTime'],
                    open=df_2020_btc['OpenPrice'],
                    high=df_2020_btc['HighPrice'],
                    low=df_2020_btc['LowPrice'],
                    close=df_2020_btc['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Bitcoin performance in 2020',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )


Here we see an increase from less than 15,000 USD to about 30,000 USD from October to December 2020.

In [ ]:
#mean price
round(df_2020_btc['ClosePrice'].mean())

In [ ]:
#median price
round(df_2020_btc['ClosePrice'].median())

In [ ]:
#lowest price
round(df_2020_btc['ClosePrice'].min())

In [ ]:
#highest price
round(df_2020_btc['ClosePrice'].max())

In [ ]:
#extract for october to calculate the growth rate from october to december
df_2020_btc_oct = df_2020_btc[df_2020_btc.Month == 10]

In [ ]:
round(((df_2020_btc['ClosePrice'].median()-df_2020_btc_oct['ClosePrice'].median())/df_2020_btc_oct['ClosePrice'].median())*100,2)

From October 2020 to December 2020, Bitcoin had a mean price of USD 18,065, median price of USD 18,110, low price of USD 11,430, highest price of USD 29,178 and a growth rate of 38.91%. 

# Analysing Bitcoin Prices for 2021

In [ ]:
df_2021 = coins[coins.Year == 2021]
df_2021_btc = df_2021[df_2021.Symbol == 'BTC']
df_2021_btc.head()

In [ ]:
#check growth for 2021
fig = go.Figure(data=go.Ohlc(x=df_2021_btc['CloseTime'],
                    open=df_2021_btc['OpenPrice'],
                    high=df_2021_btc['HighPrice'],
                    low=df_2021_btc['LowPrice'],
                    close=df_2021_btc['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Bitcoin performance in 2021',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )


Here bitcoin price fluctuates between 30,000 USD to almost 70,000 USD. From January 2021 to April 2021, there was a steady climb to about 65,000 USD. From May 2021 to July 2021, there was a drop to 30,000 USD. From August 2021 to September 2021, it increased to 50,000 USD and increased to almost 70,000 USD from October to November 2021. It dropped down to 48,000 USD and ranges between 48,000 USD - 40,000 USD in December 2021.

In [ ]:
#mean price
round(df_2021_btc['ClosePrice'].mean())

In [ ]:
#median price
round(df_2021_btc['ClosePrice'].median())

In [ ]:
#lowest price
round(df_2021_btc['ClosePrice'].min())

In [ ]:
#highest price
round(df_2021_btc['ClosePrice'].max())

In [ ]:
#extract for january to calculate the growth rate from january to december
df_2021_btc_jan = df_2021_btc[df_2021_btc.Month == 1]

In [ ]:
round(((df_2021_btc['ClosePrice'].median()-df_2021_btc_jan['ClosePrice'].median())/df_2021_btc_jan['ClosePrice'].median())*100,2)

From January 2021 to December 2021, Bitcoin had a mean price of 47,387 USD, median price of 47,943 USD, low price of 28,993 USD, highest price of 68,627 USD and a growth rate of 40.03%. 

# Analyzing bitcoin prices for 2022

In [ ]:
df_2022 = coins[coins.Year == 2022]
df_2022_btc = df_2022[df_2022.Symbol == 'BTC']
df_2022_btc.head()

In [ ]:
#check growth for 2021
fig = go.Figure(data=go.Ohlc(x=df_2022_btc['CloseTime'],
                    open=df_2022_btc['OpenPrice'],
                    high=df_2022_btc['HighPrice'],
                    low=df_2022_btc['LowPrice'],
                    close=df_2022_btc['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Bitcoin performance in 2022',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )


Here bitcoin price fluctuates between 45,000 USD to 35,000 USD from January 2022 to March 2022. There is either a steady decrease or a steady increase.

In [ ]:
#mean price
round(df_2022_btc['ClosePrice'].mean())

In [ ]:
#median price
round(df_2022_btc['ClosePrice'].median())

In [ ]:
#lowest price
round(df_2022_btc['ClosePrice'].min())

In [ ]:
#highest price
round(df_2022_btc['ClosePrice'].max())

In [ ]:
#extract for january to calculate the growth rate from january to december
df_2022_btc_jan = df_2022_btc[df_2022_btc.Month == 1]

In [ ]:
round(((df_2022_btc['ClosePrice'].median()-df_2022_btc_jan['ClosePrice'].median())/df_2022_btc_jan['ClosePrice'].median())*100,2)

From January 2022 to present, Bitcoin has a mean price of 40,973 USD, median price of 41,729 USD, low price of 33,321 USD, highest price of 47,777 USD and a growth rate of -0.47%. 

# Analyzing bitcoin prices from 2020 - present

In [ ]:
df_all_btc = coins[coins.Symbol == 'BTC']
df_all_btc.head()

In [ ]:
#check growth
fig = go.Figure(data=go.Ohlc(x=df_all_btc['CloseTime'],
                    open=df_all_btc['OpenPrice'],
                    high=df_all_btc['HighPrice'],
                    low=df_all_btc['LowPrice'],
                    close=df_all_btc['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Bitcoin performance from 2020 - present',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )

From October 2020 to present, Bitcoin prices ranged from 10,000 USD to up to 70,000 USD with a steady increase or steady decrease.

In [ ]:
#mean price
round(df_all_btc['ClosePrice'].mean())

In [ ]:
#median price
round(df_all_btc['ClosePrice'].median())

In [ ]:
#lowest price
round(df_all_btc['ClosePrice'].min())

In [ ]:
#highest price
round(df_all_btc['ClosePrice'].max())

In [ ]:
#use extracted data for october to calculate the growth rate from October 2020 to present
df_2020_btc = df_all_btc[df_all_btc.Year == 2020]

In [ ]:
#use extracted data for october to calculate the growth rate from October 2020 to present
round(((df_all_btc['ClosePrice'].median()-df_2020_btc_oct['ClosePrice'].median())/df_2020_btc_oct['ClosePrice'].median())*100,2)

From October 2020 to present, Bitcoin has a mean price of 42,246 USD, median price of 43,514 USD, low price of 11,430 USD, highest price of 68,627 USD and a growth rate of 233.76%. 

# Analyzing ethereum prices for 2020

In [ ]:
df_2020 = coins[coins.Year == 2020]
df_2020_eth = df_2020[df_2020.Symbol == 'ETH']
df_2020_eth.head()

In [ ]:
#check growth for 2020
fig = go.Figure(data=go.Ohlc(x=df_2020_eth['CloseTime'],
                    open=df_2020_eth['OpenPrice'],
                    high=df_2020_eth['HighPrice'],
                    low=df_2020_eth['LowPrice'],
                    close=df_2020_eth['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Ethereum performance in 2020',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )


Here we see an increase from less than 400 USD to about 700 USD from October to December 2020.

In [ ]:
#mean price
round(df_2020_eth['ClosePrice'].mean())

In [ ]:
#median price
round(df_2020_eth['ClosePrice'].median())

In [ ]:
#lowest price
round(df_2020_eth['ClosePrice'].min())

In [ ]:
#highest price
round(df_2020_eth['ClosePrice'].max())

In [ ]:
#extract for october to calculate the growth rate from october to december
df_2020_eth_oct = df_2020_eth[df_2020_eth.Month == 10]

In [ ]:
round(((df_2020_eth['ClosePrice'].median()-df_2020_eth_oct['ClosePrice'].median())/df_2020_eth_oct['ClosePrice'].median())*100,2)

From October 2020 to December 2020, Ethereum had a mean price of USD 523, median price of USD 538, low price of USD 368, highest price of USD 753 and a growth rate of 37.25%. 

# Analysing Ethereum Prices for 2021

In [ ]:
df_2021 = coins[coins.Year == 2021]
df_2021_eth = df_2021[df_2021.Symbol == 'ETH']
df_2021_eth.head()

In [ ]:
#check growth for 2021
fig = go.Figure(data=go.Ohlc(x=df_2021_eth['CloseTime'],
                    open=df_2021_eth['OpenPrice'],
                    high=df_2021_eth['HighPrice'],
                    low=df_2021_eth['LowPrice'],
                    close=df_2021_eth['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Ethereum performance in 2021',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )


Here ethereum prices fluctuate between 1,000 USD to almost 5,000 USD. From January 2021 to May 2021, there was a steady climb to about 4,000 USD. From May 2021 to July 2021, there was a drop to 1,800 USD. From August 2021 to September 2021, it increased to 4,000 USD and increased to almost 5,000 USD in November 2021. It dropped down to 4,000 USD in December 2021.

In [ ]:
#mean price
round(df_2021_eth['ClosePrice'].mean())

In [ ]:
#median price
round(df_2021_eth['ClosePrice'].median())

In [ ]:
#lowest price
round(df_2021_eth['ClosePrice'].min())

In [ ]:
#highest price
round(df_2021_eth['ClosePrice'].max())

In [ ]:
#extract for january to calculate the growth rate from january to december
df_2021_eth_jan = df_2021_eth[df_2021_eth.Month == 1]

In [ ]:
round(((df_2021_eth['ClosePrice'].median()-df_2021_eth_jan['ClosePrice'].median())/df_2021_eth_jan['ClosePrice'].median())*100,2)

From January 2021 to December 2021, Ethereum had a mean price of 2,772 USD, median price of 2,615 USD, low price of 727 USD, highest price of 4849 USD and a growth rate of 112.83%. 

# Analyzing ethereum prices for 2022

In [ ]:
df_2022 = coins[coins.Year == 2022]
df_2022_eth = df_2022[df_2022.Symbol == 'ETH']
df_2022_eth.head()

In [ ]:
#check growth for 2021
fig = go.Figure(data=go.Ohlc(x=df_2022_eth['CloseTime'],
                    open=df_2022_eth['OpenPrice'],
                    high=df_2022_eth['HighPrice'],
                    low=df_2022_eth['LowPrice'],
                    close=df_2022_eth['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Ethereum performance in 2022',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )


Here ethereum prices fluctuate between 3,500 USD to 2,500 USD from January 2022 to present. There is either a steady decrease or a steady increase.

In [ ]:
#mean price
round(df_2022_eth['ClosePrice'].mean())

In [ ]:
#median price
round(df_2022_eth['ClosePrice'].median())

In [ ]:
#lowest price
round(df_2022_eth['ClosePrice'].min())

In [ ]:
#highest price
round(df_2022_eth['ClosePrice'].max())

In [ ]:
#extract for january to calculate the growth rate from january to december
df_2022_eth_jan = df_2022_eth[df_2022_eth.Month == 1]

In [ ]:
round(((df_2022_eth['ClosePrice'].median()-df_2022_eth_jan['ClosePrice'].median())/df_2022_eth_jan['ClosePrice'].median())*100,2)

From January 2022 to present, Ethereum has a mean price of 2,967 USD, median price of 2,960 USD, low price of 2,184 USD, highest price of 3,869 USD and a growth rate of -6.22%. 

# Analyzing ethereum prices from 2020 - present

In [ ]:
df_all_eth = coins[coins.Symbol == 'ETH']
df_all_eth.head()

In [ ]:
#check growth
fig = go.Figure(data=go.Ohlc(x=df_all_eth['CloseTime'],
                    open=df_all_eth['OpenPrice'],
                    high=df_all_eth['HighPrice'],
                    low=df_all_eth['LowPrice'],
                    close=df_all_eth['ClosePrice']))

fig.update_layout(template='plotly_dark',
        title='Ethereum performance from 2020 - present',
        xaxis_title="Date",
        yaxis_title='Price',title_x=0.5,
        )
fig.update_yaxes( # the y-axis is in dollars
        tickprefix="$", showgrid=False
    )

fig.update_xaxes(
         showgrid=True
    )

From October 2020 to present, Ethereum prices ranged from 1,000 USD to up to 5,000 USD with a steady increase or steady decrease.

In [ ]:
#mean price
round(df_all_eth['ClosePrice'].mean())

In [ ]:
#median price
round(df_all_eth['ClosePrice'].median())

In [ ]:
#lowest price
round(df_all_eth['ClosePrice'].min())

In [ ]:
#highest price
round(df_all_eth['ClosePrice'].max())

In [ ]:
#use extracted data for october to calculate the growth rate from October 2020 to present
round(((df_all_eth['ClosePrice'].median()-df_2020_eth_oct['ClosePrice'].median())/df_2020_eth_oct['ClosePrice'].median())*100,2)

From October 2020 to present, Ethereum has a mean price of 2,460 USD, median price of 2,480 USD, low price of 368 USD, highest price of 4,849 USD and a growth rate of 532.65%. 

# Volume of Trade

In [ ]:
#check volume for bitcoin
# creating trace1
trace1 =go.Scatter(
                    x = df_2020_btc.ClosePrice,
                    y = df_2020_btc.Volume,
                    mode = "markers",
                    name = "2020",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text= df_2020_btc.CloseTime)
# creating trace2
trace2 =go.Scatter(
                    x = df_2021_btc.ClosePrice,
                    y = df_2021_btc.Volume,
                    mode = "markers",
                    name = "2021",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text= df_2021_btc.CloseTime)
# creating trace3
trace3 =go.Scatter(
                    x = df_2022_btc.ClosePrice,
                    y = df_2022_btc.Volume,
                    mode = "markers",
                    name = "2022",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text= df_2022_btc.CloseTime)
data = [trace1, trace2, trace3]
layout = dict(template='plotly_dark', title = 'Volume of Bitcoin Trade in 2020, 2021 and 2022',
              xaxis= dict(title= 'ClosePrice',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Volume',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)

The volume of bitcoin trade from 2020 to present ranges from 0-8,000 with a daily trade of about 2,000

In [ ]:
#check volume for ethereum
# creating trace1
trace1 =go.Scatter(
                    x = df_2020_eth.ClosePrice,
                    y = df_2020_eth.Volume,
                    mode = "markers",
                    name = "2020",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text= df_2020_eth.CloseTime)
# creating trace2
trace2 =go.Scatter(
                    x = df_2021_eth.ClosePrice,
                    y = df_2021_eth.Volume,
                    mode = "markers",
                    name = "2021",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text= df_2021_eth.CloseTime)
# creating trace3
trace3 =go.Scatter(
                    x = df_2022_eth.ClosePrice,
                    y = df_2022_eth.Volume,
                    mode = "markers",
                    name = "2022",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text= df_2022_eth.CloseTime)
data = [trace1, trace2, trace3]
layout = dict(template='plotly_dark', title = 'Volume of Ethereum Trade in 2020, 2021 and 2022',
              xaxis= dict(title= 'ClosePrice',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Volume',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
iplot(fig)

The volume of ethereum trade from 2020 to present ranges from 0-70k with a daily trade of 0-10k

# Conclusion

Regardless of whether the price is high or low, Bitcoin and Ethereum are the biggest cryptocurrencies at the moment known to always make a comeback. It might be hard for a novice trader who probably wants to start their trading journey this year or in the future. With the data presented above, it is safe to make a bet on these two and be patient, especially with Ethereum with a reported growth rate of 532.65%.